In [3]:
import os
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from bs4 import BeautifulSoup

In [4]:
# Filepaths
covid_data = os.path.join("data","cdc_covid.csv")

# Read csv into DataFrame
data = pd.read_csv(covid_data)
data.head()

,State/Territory,Total Cases,Confirmed Cases,Probable Cases,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Confirmed Deaths,Probable Deaths,Deaths in Last 7 Days,Death Rate per 100000,Case Rate per 100000 in Last 7 Days,Death Rate per 100K in Last 7 Days
0,Alaska,12118,NaN,NaN,1569,1643,68,NaN,NaN,2,9,30.4,0.0
1,Alabama,180916,155915.0,25001.0,10542,3701,2859,2674.0,185.0,73,58,30.8,0.2
2,Arkansas,104135,NaN,NaN,6596,3455,1782,NaN,NaN,117,59,31.3,0.6
3,American Samoa,0,NaN,NaN,0,0,0,NaN,NaN,0,0,0.0,0.0
4,Arizona,235882,230488.0,5394.0,6396,3289,5865,5578.0,287.0,59,81,12.7,0.1


In [5]:
new_data = data.drop(columns = ['Cases in Last 7 Days', 'Case Rate per 100000', 'Confirmed Deaths', 'Probable Deaths', 'Deaths in Last 7 Days', 'Death Rate per 100000', 'Case Rate per 100000 in Last 7 Days', 'Death Rate per 100K in Last 7 Days'])
new_data.head()

,State/Territory,Total Cases,Confirmed Cases,Probable Cases,Total Deaths
0,Alaska,12118,NaN,NaN,68
1,Alabama,180916,155915.0,25001.0,2859
2,Arkansas,104135,NaN,NaN,1782
3,American Samoa,0,NaN,NaN,0
4,Arizona,235882,230488.0,5394.0,5865


In [6]:
url = "https://covidtracking.com/data/national"
response = requests.get(url)

In [7]:
soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <style data-href="/styles.90cc1b6c4290fc98717a.css">
   .ddf6{box-sizing:border-box;display:flex;flex:0 1 auto;flex-direction:row;flex-wrap:wrap;margin-left:1.5rem;margin-right:1.5rem}@media (min-width:768px){.ddf6{margin-left:3.5rem;margin-right:3.5rem}}@media (min-width:1080px){.ddf6{margin-left:2rem;margin-right:2rem}}@media (min-width:1200px){.ddf6{max-width:71.25rem;margin-left:auto;margin-right:auto}}.d3a4{box-sizing:border-box;flex:0 0 auto;padding-left:.5rem;padding-right:.5rem;flex-basis:100%;max-width:100%}.d3a4:first-child{padding-left:0!important}.d3a4:last-child{padding-right:0!important}@media (min-width:768px){.d3a4{flex-basis:100%;max-width:100%;padding-left:.5rem;padding-right:.5rem}}@media (min-width:1080px){.d3a4{flex-basis:100%;max-width:100%;padding-lef

In [31]:
# results = soup.find_all('td', class_="")
#rint(len(results))
table_headers = soup.find_all("th", scope="col")
# table_rows = soup.find_all("td", class_ = "")
table_data = soup.find_all("td")
print(len(table_headers))
print(len(table_rows))


10
2493


In [32]:
# print(results[0].prettify())
print(table_data[0].prettify())

<td class="b6f7">
 <span class="_0d5c">
  Date
 </span>
 <span>
  Oct 24, 2020
 </span>
</td>



In [25]:
# Loop and append to list
table_headers_stripped = []
for i in range (0, len(table_headers)):
    table_column = table_headers[i].text.strip()
    table_headers_stripped.append(table_column)

In [27]:
# Review table headers
table_headers_stripped

['Date',
 'State (or territory)',
 'New tests',
 'Cases (confirmed plus probable)',
 'Negative PCR tests (people)',
 'Cumulative hospitalized/Ever hospitalized',
 'Currently hospitalized/Now hospitalized',
 'Deaths (confirmed and probable)',
 'Recovered',
 'Total test results']